In [157]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
from keras import utils
import tensorflow as tf

In [158]:
df = pd.read_csv('Churn_Modelling.csv')

In [159]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [160]:
# Devemos retirar aquelas colunas que não serão úteis para nosso modelo, sendo elas as 
# colunas de dados categóricos, IDs ou que não vemos necessidade de uma transformação imediata
df = df.drop(columns = ['RowNumber', 'CustomerId', 'Surname', 'Geography'])

In [161]:
df.isnull().sum()

CreditScore        0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [162]:
# Agora vamos transformar a única coluna de dados object que sobrou 'Gender'
df.loc[df['Gender'] == 'Female', 'Gender'] = 0
df.loc[df['Gender'] == 'Male', 'Gender'] = 1

In [163]:
df.head(10)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,42,2,0.00,1,1,1,101348.88,1
1,608,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,39,1,0.00,2,0,0,93826.63,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0
5,645,1,44,8,113755.78,2,1,0,149756.71,1
6,822,1,50,7,0.00,2,1,1,10062.80,0
7,376,0,29,4,115046.74,4,1,0,119346.88,1
8,501,1,44,4,142051.07,2,0,1,74940.50,0
9,684,1,27,2,134603.88,1,1,1,71725.73,0


In [164]:
# Separamos agora os dados descritivos das classes
X=df.iloc[:,:9].values
y=df.iloc[:,9].values
X

array([[619, 0, 42, ..., 1, 1, 101348.88],
       [608, 0, 41, ..., 0, 1, 112542.58],
       [502, 0, 42, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 36, ..., 0, 1, 42085.58],
       [772, 1, 42, ..., 1, 0, 92888.52],
       [792, 0, 28, ..., 1, 0, 38190.78]], dtype=object)

In [165]:
y

array([1, 0, 1, ..., 1, 1, 0])

O que podemos interpretar das duas linahs de código anteriores?
Basicamente dividimos nosso dataset em duas variáveis:
    X: Que representa os dados que vamos fornecer para que cheguemos a uma resposta, eles são dispostos em um array de 9 dimensoes.
    y: São os dados que buscamos ou seja, se ele saiu ou não daquele banco e esta representado em um array de 0 dimensões.

In [166]:
print("Shape of X",X.shape)
print("Shape of y",y.shape)

Shape of X (10000, 9)
Shape of y (10000,)


In [167]:
# Dividir os dados para treinamento, validação e teste
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import numpy as np

# Normalizar os dados
X_normalized = normalize(X, axis=0)

'''
70% - conjunto de treinamento
20% - conjunto de validação
10% - conjunto de teste
'''
X_train_val, X_test, y_train_val, y_test = train_test_split(X_normalized, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.22, random_state=42)

# Transformar os rótulos em categorias
num_classes = 2
y_train = utils.to_categorical(y_train, num_classes)
y_val = utils.to_categorical(y_val, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

Acima é feita a normalização dos dados contidos em X para que haja uma melhor performance do modelo a ser treinado, uma vez que os dados passam a estar em uma escala mais similar.

Os rótulos são então transformados em dados categóricos, que são usados para problemas de classificação como o nosso.

In [168]:
# Construindo a arquitetura do MLP
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(16, input_dim = 9, activation = 'relu'))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))

model.add(tf.keras.layers.Dense(2, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.summary()

/opt/homebrew/Cellar/jupyterlab/4.1.6_1/libexec/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_65 (Dense)                │ (None, 16)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_66 (Dense)                │ (None, 64)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_67 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_68 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,538 (21.63 KB)

 Trainable params: 5,538 (21.63 KB)

 Non-trainable params: 0 (0.00 B)

Acima construimos nossa rede neural utilizando a biblioteca tensorflow, criando tres camadas de processamento e uma de classificação.

Utilizamos um input_dim de 9 pois possuimos 9 dimensões de entrada de dados distintas.

Nossa função de ativação se divide em duas:
    relu: Função comunmente usada em redes neurais para introduzir uma nao lineariedade, ajudando a mitigar problemas como o do gradiente desvanecente.
    softmax: Função utilizada ao final de uma rede neural para transformar outputs em um vetor de probabilidades.

Por fim, definimos nossas funções de perda (loss), optimização (optimizer) e metrics
    loss = 'categorical_crossentropy': nos diz sobre a diferença entre as predições e aos rótulos;
    optimizer = 'adam': método baseado em gradiente descendente, que tem como objetivo minimizar a loss function durante o treinamento da rede neural;
    metrics = 'accuracy': mede o quão correto o classificador prediz o resultado.

In [169]:
model.fit(X_train,
         y_train,
         validation_split = 0.2,
         batch_size = 16,
         epochs = 50,
         verbose = 1,     
)

Epoch 1/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 395us/step - accuracy: 0.8003 - loss: 0.5404 - val_accuracy: 0.8063 - val_loss: 0.4899
Epoch 2/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 265us/step - accuracy: 0.7967 - loss: 0.5027 - val_accuracy: 0.8063 - val_loss: 0.4869
Epoch 3/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 262us/step - accuracy: 0.7974 - loss: 0.4966 - val_accuracy: 0.8063 - val_loss: 0.4778
Epoch 4/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 264us/step - accuracy: 0.7928 - loss: 0.4960 - val_accuracy: 0.8063 - val_loss: 0.4788
Epoch 5/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 261us/step - accuracy: 0.8002 - loss: 0.4750 - val_accuracy: 0.8063 - val_loss: 0.4642
Epoch 6/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 261us/step - accuracy: 0.7980 - loss: 0.4731 - val_accuracy: 0.8063 - val_loss: 0.4718
Epoch 7/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 261us/step - accuracy: 0.8028 - loss: 0.4565 - val_accuracy: 0.8077 - val_loss: 0.4576
Epoch 8/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 263us/step - accuracy: 0.7898 - loss: 0.4791 - 

In [170]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
# Fazer previsões nos dados de teste
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calcular a matriz de confusão
matriz_confusao = confusion_matrix(np.argmax(y_test, axis=1), y_pred_classes)

# Calcular a acurácia
acuracia = accuracy_score(np.argmax(y_test, axis=1), y_pred_classes)

# Calcular a pontuação F1
f1 = f1_score(np.argmax(y_test, axis=1), y_pred_classes, average='macro')

print("Matriz de Confusão:")
print(matriz_confusao)
print("Acurácia:", acuracia)
print("Pontuação F1:", f1)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 480us/step
Matriz de Confusão:
[[775  34]
 [104  87]]
Acurácia: 0.862
Pontuação F1: 0.7379693765949691


Do código e do resultado acima podem tirar o seguinte:
Nossa acurácia está boa, 86%, no entanto ainda temos muitos erros de classificação, sendo 138 situações classificadas erroneamente.

Podemos tentar melhorar essas condições utilizando da coluna Geography, tranformando-a em dados rotulados.

In [171]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [172]:
df = df.drop(columns = ['RowNumber', 'CustomerId', 'Surname'])

In [173]:
print(df['Geography'].unique())

['France' 'Spain' 'Germany']


In [174]:
#France = 0
#Spain = 1
#Germany = 2
df.loc[df['Geography'] == 'France', 'Geography'] = 0
df.loc[df['Geography'] == 'Spain', 'Geography'] = 1
df.loc[df['Geography'] == 'Germany', 'Geography'] = 2
df.loc[df['Gender'] == 'Female', 'Gender'] = 0
df.loc[df['Gender'] == 'Male', 'Gender'] = 1

In [175]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,1,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,1,0,43,2,125510.82,1,1,1,79084.10,0


In [176]:
X=df.iloc[:,:10].values
y=df.iloc[:,10].values

In [177]:
print("Shape of X",X.shape)
print("Shape of y",y.shape)

Shape of X (10000, 10)
Shape of y (10000,)


In [178]:
# Normalizar os dados
X_normalized = normalize(X, axis=0)

'''
70% - conjunto de treinamento
20% - conjunto de validação
10% - conjunto de teste
'''
X_train_val, X_test, y_train_val, y_test = train_test_split(X_normalized, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.22, random_state=42)

# Transformar os rótulos em categorias
num_classes = 2
y_train = utils.to_categorical(y_train, num_classes)
y_val = utils.to_categorical(y_val, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [179]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(16,input_dim=10,activation='relu'))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dense(64,activation='relu'))

#model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

/opt/homebrew/Cellar/jupyterlab/4.1.6_1/libexec/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_69 (Dense)                │ (None, 16)             │           176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_70 (Dense)                │ (None, 64)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_71 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_72 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,554 (21.70 KB)

 Trainable params: 5,554 (21.70 KB)

 Non-trainable params: 0 (0.00 B)

In [180]:
model.fit(X_train,
          y_train,
          validation_split=0.2,
          batch_size=16,
          epochs=50,
          verbose=1,
          class_weight=class_weights)

Epoch 1/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 404us/step - accuracy: 0.8028 - loss: 0.5328 - val_accuracy: 0.8063 - val_loss: 0.4892
Epoch 2/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 260us/step - accuracy: 0.8023 - loss: 0.4939 - val_accuracy: 0.8063 - val_loss: 0.4758
Epoch 3/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 268us/step - accuracy: 0.7971 - loss: 0.4864 - val_accuracy: 0.8063 - val_loss: 0.4583
Epoch 4/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 274us/step - accuracy: 0.7986 - loss: 0.4678 - val_accuracy: 0.8063 - val_loss: 0.4597
Epoch 5/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 270us/step - accuracy: 0.7969 - loss: 0.4591 - val_accuracy: 0.8063 - val_loss: 0.4498
Epoch 6/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 274us/step - accuracy: 0.8004 - loss: 0.4540 - val_accuracy: 0.8063 - val_loss: 0.4473
Epoch 7/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 275us/step - accuracy: 0.7939 - loss: 0.4634 - val_accuracy: 0.8063 - val_loss: 0.4455
Epoch 8/50
351/351 ━━━━━━━━━━━━━━━━━━━━ 0s 270us/step - accuracy: 0.8022 - loss: 0.4437 - 

In [181]:
# Fazer previsões nos dados de teste
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calcular a matriz de confusão
matriz_confusao = confusion_matrix(np.argmax(y_test, axis=1), y_pred_classes)

# Calcular a acurácia
acuracia = accuracy_score(np.argmax(y_test, axis=1), y_pred_classes)

# Calcular a pontuação F1
f1 = f1_score(np.argmax(y_test, axis=1), y_pred_classes, average='macro')

print("Matriz de Confusão:")
print(matriz_confusao)
print("Acurácia:", acuracia)
print("Pontuação F1:", f1)


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step
Matriz de Confusão:
[[793  16]
 [151  40]]
Acurácia: 0.833
Pontuação F1: 0.6143106900605324


Dessa vez não conseguimos obter um resultado melhor.
Nosso F1 score poderia ter sido melhor, uma vez que ele nos diz a precisão do nosso modelo, sendo ideal no minimo 0.70.
Há diversas maneiras para tentarmos melhorar nossa rede neural para prover uma melhor classificação como:
    Mudar a quantidade de camadas escondidas
    Alterar as funções de ativação
    Melhorar a qualidade dos dados
    Aumentar ou diminuir a quantidade de epochs